In [1]:
import json

with open("tags_.json", 'r') as f:
    data = [
        json.loads(x) for x in f.readlines()
    ]

In [ ]:
import os

# structure:
# path to parent folder, name, contents
plain_text_files = list()
plain_text_file_path = "test_input/test_repo_/"

file_types_to_consider = ["txt", 'md', 'yml']

def walk_dir(path, ind=0):
    for name in os.listdir(path):
        if os.path.isdir(path + name + '/'):
            walk_dir(path + name + '/', ind+4)
        else:
            ext = name.split('.')[-1]
            if ext in file_types_to_consider:
                with open(path + name, 'r', encoding="utf-8") as f:
                    contents = f.read()
                    full_path = path + name
                plain_text_files.append(
                    (full_path[len(plain_text_file_path):], contents)
                )
                

walk_dir(plain_text_file_path)

In [ ]:
plain_text_files

In [ ]:
# list unique files

unique_files = set()
for item in data:
    unique_files.add(item['rel_fname'])
for item in plain_text_files:
    unique_files.add(item[0])

unique_files = ["root/"+x for x in list(unique_files)]
unique_files

In [ ]:
nodes_csv = {"root": 0}
edges_csv = set()

def extract_relationships(file_paths):
    """
    Extracts unique relationships from a list of file paths.

    Args:
    - file_paths (list of str): A list of file paths.

    Returns:
    - list of str: A list of relationships in the format "folder contains folder" or "folder contains file".
    """
    relationships = set()

    for path in file_paths:
        # Split the path into components
        parts = path.split('/')

        # Generate relationships
        for i in range(1, len(parts)):
            start = 1 if i > 1 else 0
            # Construct the current folder path
            current_folder = '/'.join(parts[start:i])
            # Construct the next part (either a folder or a file)
            next_part = '/'.join(parts[1:i+1])
            
            for folder in [current_folder, next_part]:
                if folder not in nodes_csv:
                    nodes_csv[folder] = len(nodes_csv)

            relation = "folder" if i < len(parts) - 1 else "file"
            relationships.add(f"{current_folder} contains {relation} {next_part}")
            edges_csv.add((
                nodes_csv[current_folder], f"contains {relation}", nodes_csv[next_part]
            ))

    return list(relationships)


relationships = extract_relationships(unique_files)
_=[print(x) for x in sorted(list(relationships))]

In [ ]:
# All def links
defs = list()
defs_dict = dict()

for item in data:
    if item['kind'] == "ref": continue
    """f
    This thing also provides those relations
    file contains function
    file contains class
    class contains method
    """
    defs.append((
        item['fname'], item['name'], item['info']
    ))
    defs_dict[
        (item['fname'], item['name'].split()[-1])
    ] = len(defs_dict)

    loc = (item['rel_fname'], item['name'])
    if loc not in nodes_csv:
        nodes_csv[loc] = len(nodes_csv)

    print(defs[-1][:-1], len(defs_dict)-1)

    if item['category'] == "function" and '.' not in item['name']:
        edges_csv.add((
            nodes_csv[item['rel_fname']], "file contains function", nodes_csv[loc]
        ))
    
    elif item['category'] == "class":
        edges_csv.add((
            nodes_csv[item['rel_fname']], "file contains class", nodes_csv[loc]
        ))
        for method in item['info'].split('\n'):
            method_loc = (item['rel_fname'], method)
            if method_loc not in nodes_csv:
                nodes_csv[method_loc] = len(nodes_csv)
            edges_csv.add((
                nodes_csv[loc], "class contains method", nodes_csv[method_loc]
            ))

In [ ]:
edges_csv

In [ ]:
import os
import ast

def parse_imports(file_path):
    with open(file_path, 'r') as file:
        tree = ast.parse(file.read())

    imports = {}
    for node in ast.walk(tree):
        if isinstance(node, ast.Import):
            for alias in node.names:
                imports[alias.asname or alias.name] = alias.name
        elif isinstance(node, ast.ImportFrom):
            module = node.module
            for alias in node.names:
                imports[alias.asname or alias.name] = f"{module}.{alias.name}"

    return imports

def find_imported_function_location(file_path, function_name):
    # Parse the imports in the file
    imports = parse_imports(file_path)

    # Get the base directory of the file
    base_dir = os.path.dirname(file_path)

    # Check if the function is in the imports dictionary
    if function_name in imports:
        import_path = imports[function_name]
        print(file_path, base_dir)
        print(import_path)
        # Resolve the import path to a file path
        parts = (parts := import_path.split('.'))[len(parts)-2:-1]
        resolved_path = os.path.join(base_dir, *parts) + '.py'
        return resolved_path
    else:
        return ""

def get_local_path(file_path, function_name):
    for item in data:
        if item['kind'] == 'def' and item['fname'] == file_path and item['name'].split('.')[-1] == function_name:
            return file_path
    
    return ""
    
def resolve_reference(file_path, function_name):
    if location := get_local_path(file_path, function_name) or find_imported_function_location(file_path, function_name):
        # return location
        return defs_dict.get(
            (location, function_name), -1
        )
    return -1


for item in data:
    if item['kind'] == "def": continue
    if item['rel_fname'] != "tests/test_utils.py" or item['name'] != "get_python_paths_list": continue
    # print(item['rel_fname'], item['name'])
    ref = resolve_reference(item['fname'], item['name'])
    # print(item['rel_fname'], item['name'], ref)

    break

In [ ]:
import ast
import os
import builtins

import ast
import os
import builtins

def resolve_reference(name, file_path, project_root):
    """
    Resolves the reference of a function, class, or method to its definition.

    Args:
    - name (str): The name of the function, class, or method to resolve.
    - file_path (str): The relative file path where the name is referenced, starting from the project root.
    - project_root (str): The root directory of the project.

    Returns:
    - str: The file path of the definition, or an empty string if it's a built-in or external module.
    """

    # Parse the file to analyze imports and definitions
    with open(os.path.join(project_root, file_path), 'r') as file:
        tree = ast.parse(file.read())

    # Track imports
    imports = {}

    for node in ast.walk(tree):
        if isinstance(node, ast.Import):
            for alias in node.names:
                imports[alias.asname or alias.name] = alias.name
        elif isinstance(node, ast.ImportFrom):
            module = node.module
            for alias in node.names:
                imports[alias.asname or alias.name] = f"{module}.{alias.name}"

    # Check if the name is imported
    if name in imports:
        import_path = imports[name]
        if '.' in import_path:
            # Handle nested imports
            module_parts = import_path.split('.')
            module_file = os.path.join(project_root, *module_parts[:-1], f"{module_parts[-2]}.py")
            if os.path.exists(module_file):
                return module_file
        else:
            # Handle top-level imports
            module_file = os.path.join(project_root, f"{import_path}.py")
            if os.path.exists(module_file):
                return module_file

    # Search the project directory for the definition
    for root, dirs, files in os.walk(project_root):
        for file in files:
            if file.endswith('.py'):
                module_file_path = os.path.join(root, file)
                with open(module_file_path, 'r') as f:
                    module_tree = ast.parse(f.read())
                    for node in ast.walk(module_tree):
                        if isinstance(node, ast.FunctionDef) or isinstance(node, ast.ClassDef):
                            if node.name == name:
                                return module_file_path

    # If the name is not found, assume it's an external module
    return "No location detected"

project_root = 'test_input/codegraph-main'

current_ref = None
for i, item in enumerate(data):
    if item['kind'] == "def":
        current_ref = item.copy()
        continue

    name = item['name']
    file_path = item['rel_fname']

    definition_location = resolve_reference(name, file_path, project_root)
    definition_location = definition_location[len(project_root)+1:]

    location_id = nodes_csv.get((definition_location, name), -1)
    
    if location_id == -1: continue

    edges_csv.add((
        nodes_csv[(current_ref['rel_fname'], current_ref['name'])],
        "references", location_id
    ))


In [ ]:
edges_csv

In [ ]:
nodes_csv

In [ ]:
plain_text_dict = {v[0]: v[1] for v in plain_text_files}
plain_text_dict

In [13]:
for item in data:
    loc = (item['rel_fname'], item['name'])
    node_id = nodes_csv.get(loc, -1)
    if node_id == -1 or item['kind'] == "ref": continue
    del nodes_csv[loc]
    nodes_csv[node_id] = f"# location: {item['rel_fname']}\n{item['info']}"

nodes_csv_items = nodes_csv.copy().items()
for k, v in nodes_csv_items:
    if type(v) == int:
        del nodes_csv[k]
        nodes_csv[v] = k

In [ ]:
nodes_csv_items = nodes_csv.copy().items()
for node_id, node_location in nodes_csv_items:
    if text := plain_text_dict.get(node_location, ""):
        nodes_csv[node_id] = f"{node_location} file contents: {text}"

nodes_csv

In [15]:
nodes = [(k, v) for k, v in nodes_csv.items()]
nodes = sorted(nodes, key=lambda l:l[0])

with open("a", 'w') as f:
    f.write("id,name\n")
    for item in nodes:
        f.write(f"{item[0]},{item[1]}\n")

# with open("a", 'w') as f:
#     f.write("id_head,type,id_tail\n")
#     for item in edges_csv:
#         f.write(f"{item[0]},{item[1]},{item[2]}\n")

In [9]:
import pandas as pd

with open("G-Retriever/dataset_/edges.csv") as f:
    edges = f.readlines()[1:]
    edges = [x.strip().split(',') for x in edges]
    heads, relation_types, tails = zip(*edges)

with open("G-Retriever/dataset_/nodes.csv") as f:
    nodes = f.readlines()[1:]
    nodes = [x.strip().split(',') for x in nodes]
    node_ids = [x[0] for x in nodes]
    nodes_texts = [','.join(x[1:]) for x in nodes]

In [ ]:
set(tails) - set(node_ids)

In [ ]:
import importlib.util
import sys

# Assuming the folder is named 'my-folder' and it contains a module 'mymodule.py'
folder_name = 'G-Retriever'
module_name = 'src/utils/lm_modeling'
module_path = f"./{folder_name}/{module_name}.py"

# Load the module
spec = importlib.util.spec_from_file_location(module_name, module_path)
module = importlib.util.module_from_spec(spec)
sys.modules[module_name] = module
spec.loader.exec_module(module)

In [3]:
# default module name
llm_module_name = "sbert"

model, tokenizer, device = module.load_model[llm_module_name]()
text2embedding = module.load_text2embedding[llm_module_name]

inherit model weights from sentence-transformers/all-roberta-large-v1


In [ ]:
from tqdm.notebook import tqdm
import torch
from torch_geometric.data.data import Data

def process_in_batches(elements, batch_size, processing_function, *args):
    results = []
    for i in tqdm(range(0, len(elements), batch_size)):
        batch_elements = elements[i:i + batch_size]
        batch_result = processing_function(*args, batch_elements)
        results.append(batch_result)
    return torch.cat(results, dim=0)

# Define batch size
batch_size = 100

# Process nodes in batches
x = process_in_batches(nodes_texts, batch_size, text2embedding, model, tokenizer, device)

# Process relation types in batches
e = process_in_batches(relation_types, batch_size, text2embedding, model, tokenizer, device)

# Create edge index tensor
edge_index = torch.LongTensor([
    pd.Series(heads).astype(int), pd.Series(tails).astype(int)
])

# Create graph data structure
data = Data(x=x, edge_index=edge_index, edge_attr=e, num_nodes=len(nodes_texts))

# Save the graph data
torch.save(data, 'test_input_/output/graph.pt')